# Smart Query
This Notebook allows user to query a random piece of information from SumDB first. If the information is found in SumDB, it will then go to LogosCluster to find the full-length article. Otherwise, return nothing.

In [1]:
%pip install psycopg2-binary rich scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 19.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 9.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import List, Tuple
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from psycopg2 import sql
from rich import print
import psycopg2


# HELPER FUNCTIONS

In [3]:
def log(message: str) -> None:
    """
    Logs a message to the console.
    """
    print(message)
    with open("sumdb_log.txt", "a") as log_file:
        log_file.write(message + "\n")

In [4]:
def get_column_names(conn, table):
    """
    Fetches and logs the column names of a specified table.
    """
    query = sql.SQL("""
        SELECT column_name 
        FROM information_schema.columns 
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """)
    cur = conn.cursor()
    cur.execute(query, (table,))
    columns = cur.fetchall()
    cur.close()
    return [col[0] for col in columns]

# Perform SumDB Query

In [5]:
# sumb db config
sumdb_topic = "logosdb-sumdb"  # using localhost for now
port = "5432"
dbname = "db"  # internal database name
username = "user"
password = "password"

table = "test"  # Name of table to query

In [6]:
# Connect to the database
conn = psycopg2.connect(
    dbname=dbname,
    user=username,
    password=password,
    host=sumdb_topic,
    port=port
)

# Format datetime for readability
formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
log(f"[{formatted_datetime}] Connected to database '{dbname}' on {sumdb_topic}:{port} as '{username}'")

# Get and log column names
column_names = get_column_names(conn, table)
log("Column names in '{}' table: {}".format(table, ", ".join(column_names)))


[2024-08-26 19:22:00] Connected to database 'db' on logosdb-sumdb:5432 as 'user'

Column names in 'test' table: id, chunkstart, chunkend, topic, summary, updatedat

## HELPER FUNCTIONS

In [7]:
def similarity_search(user_query: str, chunk_summary: str) -> float:
    """
    Calculate the similarity between the user query and a chunk summary using cosine similarity.
    """
    # Create a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Combine the user query and chunk summary into a list
    documents = [user_query, chunk_summary]

    # Fit and transform the documents into TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform(documents)

    # Calculate the cosine similarity between the first and second document
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    # Return the similarity score as a float
    return similarity[0][0]

In [8]:
def get_all_sumdb_data(conn: psycopg2.extensions.connection, table: str) -> List[Tuple[str]]:
    """
    Fetches all data from the specified table.
    """
    query = sql.SQL("SELECT chunkstart, chunkend, topic, summary, updatedat FROM {}").format(sql.Identifier(table))
    cur = conn.cursor()
    cur.execute(query)
    data = cur.fetchall()
    cur.close()
    return data

In [9]:
def get_relevant_vectors_from_summary(chunk_summary: str, user_query: str, threshold: float=0.5) -> List[Tuple[str, float]]:
    """
    Find all vectors (rows) in the chunk summary that have a similarity score above the threshold with the user query.
    """
    
    relevant_rows = []
    current_row = ""

    # Iterate over each character in the chunk summary
    for char in chunk_summary:
        # Split summary into rows by newline symbol
        if char != '\n':
            current_row += char
            continue

        # Skip rows that are too short
        current_row = current_row.lower().strip()
        if len(current_row) <= 5:
            current_row = ""
            continue

        score = similarity_search(user_query, current_row)

        # only add rows with a similarity score above the threshold
        if score > threshold:
            relevant_rows.append((current_row, score))

        current_row = ""

    
    # Check the last row if it doesn't end with a newline
    current_row = current_row.lower().strip()
    if len(current_row) > 5:
        score = similarity_search(user_query, current_row)
        if score > threshold:
            relevant_rows.append((current_row, score))
    
    return relevant_rows

In [10]:
def get_relevant_vectors_of_all_chunk(raw_data: List[Tuple[str]], user_query: str, threshold: float=0.5) -> List[Tuple[float, str, int, int, str]]:
    """
    Get all relevant vectors from all chunks in the raw data.
    (which has similarity > threshold)
    """

    # Do similarity search for each chunk summary
    relevant_vectors = []
    
    progress = 0 # out of 100%
    # A chunk is considered relevant if 1 row within it is relevant
    for i, chunk_row in enumerate(raw_data):
        if i % (len(raw_data) // 10) == 0 and i != 0:
            progress += 10
            log(f"Progress: {progress}%, Sum Chunk: {i}/{len(raw_data)}")
        chunk_start, chunk_end, topic, chunk_summary = chunk_row[:4]
        valid_rows = get_relevant_vectors_from_summary(chunk_summary, user_query, threshold)
        
        # Add valid vectors from chunk to relevant_vectors
        for row, score in valid_rows:
            relevant_vectors.append((score, row, chunk_start, chunk_end, topic))

    return relevant_vectors

In [25]:
def find_detail_articles(relevant_vectors: List[Tuple[float, str, int, int, str]], k_docs: int=5) -> List[Tuple[float, str, str, int, int, str]]:
    """
    Fetches the detailed articles for the top-k most relevant vectors.
    """
    logos_dbname = "db"  # internal database name
    logos_username = "user"
    logos_password = "password"

    logos_table = "test"
    output = []

    # sorted by similarity score
    relevant_vectors.sort(key=lambda x: x[0], reverse=True)
    for vect in relevant_vectors[:k_docs]:
        score, row, chunk_start, chunk_end, node_topic = vect
        row_id = int(row.split(".")[0]) # extract row id from row string

        # connect to db first
        logos_conn = psycopg2.connect(
            dbname=logos_dbname,
            user=logos_username,
            password=logos_password,
            host=node_topic,
        )
        
        # then do query on topic node
        query = sql.SQL("""
            SELECT question, answer, keywords FROM {} WHERE id = %s
        """).format(sql.Identifier(logos_table))
        
        with logos_conn.cursor() as cur:
            cur.execute(query, (row_id,))
            articles = cur.fetchall()
            output.append((score, row, articles[0][1], chunk_start, chunk_end, node_topic))
            
        logos_conn.close()

    return output

## MAIN FUNCTION

In [12]:
log("Fetching all data from the SumDB")
raw_data = get_all_sumdb_data(conn, table)
log("Data fetched successfully.")

Fetching all data from the SumDB

Data fetched successfully.

In [13]:
# Get Relevant Chunks
user_query = r"datamaker is a canadian developer and marketer of test data management software datamaker was founded by mathieu pelletier in 2020"
threshold = 0.5
relevant_vectors = get_relevant_vectors_of_all_chunk(raw_data, user_query, threshold)

Progress: 10%, Sum Chunk: 45/450

Progress: 20%, Sum Chunk: 90/450

Progress: 30%, Sum Chunk: 135/450

Progress: 40%, Sum Chunk: 180/450

Progress: 50%, Sum Chunk: 225/450

Progress: 60%, Sum Chunk: 270/450

Progress: 70%, Sum Chunk: 315/450

Progress: 80%, Sum Chunk: 360/450

Progress: 90%, Sum Chunk: 405/450

In [14]:
if not relevant_vectors:
    log(f"No relevant vectors found for query '{user_query}'")
else:
    log(f"Found {len(relevant_vectors)} relevant vectors for query '{user_query}'")
    log(f"Print first 5 relevant vectors:")
    for i, chunk in enumerate(relevant_vectors[:5]):
        score, row, chunk_start, chunk_end, topic = chunk
        log(f"Relevant Vector {i+1}:")
        log(f"Topic: {topic}, Score: {score}")
        log(f"Vector Content: {row}")

Found 1 relevant vectors for query 'datamaker is a canadian developer and marketer of test data management software
datamaker was founded by mathieu pelletier in 2020'

Print first 5 relevant vectors:

Relevant Vector 1:

Topic: Computer-Science, Score: 0.5827386398810945

Vector Content: 224501. datamaker datamaker is a canadian developer and marketer of test data management software 
datamaker was founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating 
synthetic data datamaker is a canadian developer and marketer of test data management software datamaker was 
founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating synthetic data 
history  the company was started in stealth mode in 2018 the idea was developed inside a big data project the 
software was publically launched in the september 2021 reasons challenges faced the company has receive a preseed 
funding of 100k  the startup is part of startup montreal ecosystem   software and services  the software is 
available in aws marketplace as an ec2 ami  there is also a offering on in azure the platform is available as a 
standalone application or in the cloud  references  categorysoftware forcetoc

In [26]:
# Get top k articles
k = 5
detail_articles = find_detail_articles(relevant_vectors, k)

print(detail_articles)

[
    (
        np.float64(0.5827386398810945),
        '224501. datamaker datamaker is a canadian developer and marketer of test data management software 
datamaker was founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating 
synthetic data datamaker is a canadian developer and marketer of test data management software datamaker was 
founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating synthetic data 
history  the company was started in stealth mode in 2018 the idea was developed inside a big data project the 
software was publically launched in the september 2021 reasons challenges faced the company has receive a preseed 
funding of 100k  the startup is part of startup montreal ecosystem   software and services  the software is 
available in aws marketplace as an ec2 ami  there is also a offering on in azure the platform is available as a 
standalone application or in the cloud  references  categorysoftware forcetoc',
        'datamaker datamaker is a canadian developer and marketer of test data management software datamaker was 
founded by mathieu pelletier in 2020  its products and services aim to provide tools for generating synthetic data 
datamaker is a canadian developer and marketer of test data management software datamaker was founded by mathieu 
pelletier in 2020  its products and services aim to provide tools for generating synthetic data  history  the 
company was started in stealth mode in 2018 the idea was developed inside a big data project the software was 
publically launched in the september 2021 reasons challenges faced the company has receive a preseed funding of 
100k  the startup is part of startup montreal ecosystem   software and services  the software is available in aws 
marketplace as an ec2 ami  there is also a offering on in azure the platform is available as a standalone 
application or in the cloud  references  categorysoftware forcetoc',
        224501,
        224853,
        'Computer-Science'
    )
]

In [ ]:
conn.close()